In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amirul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amirul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\amirul\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
documents = [
 "Rafael Nadal Joins Roger Federer in Missing U.S. Open",
 "Rafael Nadal Is Out of the Australian Open",
 "Biden Announces Virus Measures",
 "Biden's Virus Plans Meet Reality",
 "Where Biden's Virus Plan Stands" ]

In [9]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalnum()]
    tokens = [token for token in tokens if token not in stop_words] 
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

preprocessed_documents = [preprocess_text(doc) for doc in documents]
preprocessed_documents

[['rafael', 'nadal', 'join', 'roger', 'federer', 'missing', 'open'],
 ['rafael', 'nadal', 'australian', 'open'],
 ['biden', 'announces', 'virus', 'measure'],
 ['biden', 'virus', 'plan', 'meet', 'reality'],
 ['biden', 'virus', 'plan', 'stand']]

In [10]:
dictionary = corpora.Dictionary(preprocessed_documents)
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]

In [11]:
lda_model = LdaModel(corpus, num_topics=2, id2word=dictionary, passes=15)

In [14]:
article_labels = []

for i, doc in enumerate(preprocessed_documents):
    bow = dictionary.doc2bow(doc)
    topics = lda_model.get_document_topics(bow)
    dominant_topic = max(topics, key=lambda x: x[1])[0]
    article_labels.append(dominant_topic)
    
df = pd.DataFrame({"Article": documents, "Topic": article_labels})

print("Table with Articles and Topic:")
print(df)
print()

Table with Articles and Topic:
                                             Article  Topic
0  Rafael Nadal Joins Roger Federer in Missing U....      0
1         Rafael Nadal Is Out of the Australian Open      0
2                     Biden Announces Virus Measures      1
3                   Biden's Virus Plans Meet Reality      1
4                    Where Biden's Virus Plan Stands      1



In [15]:
print("Top Terms for Each Topic:")
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}:")
    terms = [term.strip() for term in topic.split("+")]
    for term in terms:
        weight, word = term.split("*")
        print(f"-{word.strip()} (weight: {weight.strip()})")
    print()

Top Terms for Each Topic:
Topic 0:
-"rafael" (weight: 0.130)
-"nadal" (weight: 0.130)
-"open" (weight: 0.130)
-"roger" (weight: 0.078)
-"join" (weight: 0.078)
-"federer" (weight: 0.078)
-"missing" (weight: 0.078)
-"australian" (weight: 0.078)
-"meet" (weight: 0.029)
-"reality" (weight: 0.029)

Topic 1:
-"virus" (weight: 0.167)
-"biden" (weight: 0.167)
-"plan" (weight: 0.118)
-"measure" (weight: 0.072)
-"announces" (weight: 0.072)
-"stand" (weight: 0.072)
-"reality" (weight: 0.070)
-"meet" (weight: 0.070)
-"australian" (weight: 0.024)
-"open" (weight: 0.024)



### Example 2

In [24]:
df = pd.read_csv('npr.csv')
documents = df['Article'].tolist()

In [26]:
stop_words = set(stopwords.words('english')) # Create a set of English stopwords
lemmatizer = WordNetLemmatizer() # Initialize a WordNet lemmatizer
def preprocess_text(text):
 tokens = word_tokenize(text.lower()) # Tokenize the text into words and convert to lowercase
 tokens = [token for token in tokens if token.isalnum()] # Filter out non-alphanumeric tokens
 tokens = [token for token in tokens if token not in stop_words] # Remove stopwords from the tokens
 tokens = [lemmatizer.lemmatize(token) for token in tokens] # Lemmatize each token
 return tokens # Return the preprocessed tokens
preprocessed_documents = [preprocess_text(doc) for doc in documents] 
print(preprocessed_documents[0])

['washington', '2016', 'even', 'policy', 'bipartisan', 'politics', 'sense', 'year', 'show', 'little', 'sign', 'ending', 'president', 'obama', 'moved', 'sanction', 'russia', 'alleged', 'interference', 'election', 'concluded', 'republican', 'long', 'called', 'similar', 'severe', 'measure', 'could', 'scarcely', 'bring', 'approve', 'house', 'speaker', 'paul', 'ryan', 'called', 'obama', 'measure', 'appropriate', 'also', 'overdue', 'prime', 'example', 'administration', 'ineffective', 'foreign', 'policy', 'left', 'america', 'weaker', 'eye', 'gop', 'leader', 'sounded', 'much', 'theme', 'urging', 'president', 'obama', 'year', 'take', 'strong', 'action', 'deter', 'russia', 'worldwide', 'aggression', 'including', 'operation', 'wrote', 'devin', 'nunes', 'chairman', 'house', 'intelligence', 'committee', 'week', 'left', 'office', 'president', 'suddenly', 'decided', 'stronger', 'measure', 'indeed', 'appearing', 'cnn', 'frequent', 'obama', 'critic', 'trent', 'frank', 'called', 'much', 'tougher', 'acti

In [28]:
dictionary = corpora.Dictionary(preprocessed_documents)
dictionary.filter_extremes(no_below=15, no_above=0.5)
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]

In [29]:
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15) 

In [30]:
article_labels = []
for i, doc in enumerate(preprocessed_documents):
 # for each document, convert to bag-of-words representation
 bow = dictionary.doc2bow(doc)
 # get list of topic probabilities
 topics = lda_model.get_document_topics(bow)
 # determine topic with highest probability
 dominant_topic = max(topics, key=lambda x: x[1])[0]
 # append to the list
 article_labels.append(dominant_topic)

df_result = pd.DataFrame({"Article": documents, "Topic": article_labels})
print("Table with Articles and Topic:")
print(df_result)
print()

Table with Articles and Topic:
                                                 Article  Topic
0      In the Washington of 2016, even when the polic...      1
1        Donald Trump has used Twitter  —   his prefe...      1
2        Donald Trump is unabashedly praising Russian...      1
3      Updated at 2:50 p. m. ET, Russian President Vl...      1
4      From photography, illustration and video, to d...      4
...                                                  ...    ...
11987  The number of law enforcement officers shot an...      0
11988    Trump is busy these days with victory tours,...      1
11989  It’s always interesting for the Goats and Soda...      2
11990  The election of Donald Trump was a surprise to...      1
11991  Voters in the English city of Sunderland did s...      0

[11992 rows x 2 columns]



In [31]:
for topic_id in range(lda_model.num_topics):
 print(f"Top terms for Topic #{topic_id}:")
 top_terms = lda_model.show_topic(topic_id, topn=10)
 print([term[0] for term in top_terms])
 print()

Top terms for Topic #0:
['police', 'country', 'report', 'city', 'government', 'state', 'attack', 'told', 'two', 'officer']

Top terms for Topic #1:
['trump', 'clinton', 'president', 'state', 'republican', 'campaign', 'election', 'obama', 'vote', 'house']

Top terms for Topic #2:
['food', 'water', 'human', 'study', 'scientist', 'world', 'animal', 'disease', 'science', 'university']

Top terms for Topic #3:
['health', 'school', 'percent', 'company', 'state', 'care', 'student', 'child', 'program', 'million']

Top terms for Topic #4:
['know', 'think', 'thing', 'life', 'woman', 'really', 'story', 'show', 'book', 'go']



In [33]:
print("Top Terms for Each Topic:")
for idx, topic in lda_model.print_topics():
 print(f"Topic {idx}:")
 terms = [term.strip() for term in topic.split("+")]
 for term in terms:
     weight, word = term.split("*")
     print(f"- {word.strip()} (weight: {weight.strip()})")
 print()


Top Terms for Each Topic:
Topic 0:
- "police" (weight: 0.007)
- "country" (weight: 0.006)
- "report" (weight: 0.005)
- "city" (weight: 0.005)
- "government" (weight: 0.005)
- "state" (weight: 0.005)
- "attack" (weight: 0.004)
- "told" (weight: 0.004)
- "two" (weight: 0.003)
- "officer" (weight: 0.003)

Topic 1:
- "trump" (weight: 0.029)
- "clinton" (weight: 0.012)
- "president" (weight: 0.010)
- "state" (weight: 0.009)
- "republican" (weight: 0.008)
- "campaign" (weight: 0.008)
- "election" (weight: 0.006)
- "obama" (weight: 0.006)
- "vote" (weight: 0.005)
- "house" (weight: 0.005)

Topic 2:
- "food" (weight: 0.007)
- "water" (weight: 0.005)
- "human" (weight: 0.003)
- "study" (weight: 0.003)
- "scientist" (weight: 0.003)
- "world" (weight: 0.003)
- "animal" (weight: 0.003)
- "disease" (weight: 0.003)
- "science" (weight: 0.003)
- "university" (weight: 0.003)

Topic 3:
- "health" (weight: 0.009)
- "school" (weight: 0.007)
- "percent" (weight: 0.007)
- "company" (weight: 0.006)
- "state